In [84]:
from datasets import load_dataset
from huggingface_hub import snapshot_download
from transformers import pipeline
import torch
import string
import json
import jellyfish
import numpy as np
import os




In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
# Retrieve template and find valid and turn datasets
snapshot_download(
    "McGill-NLP/WebLINX", repo_type="dataset", allow_patterns="templates/*", local_dir="."
)
with open('templates/llama.txt') as f:
    template = f.read()
valid_dataset = load_dataset("McGill-NLP/weblinx", split="validation")
valid_turn_text = []
valid_turn_info = []
for i in valid_dataset:
    valid_turn_text.append(template.format(**i))
    valid_turn_info.append(i)

test_dataset = load_dataset("McGill-NLP/weblinx", split="test")
test_turn_text = []
test_turn_info = []
for i in test_dataset:
    test_turn_text.append(template.format(**i))
    test_turn_info.append(i)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/McGill-NLP/WebLINX/revision/main HTTP/1.1" 200 2412
Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/McGill-NLP/WebLINX/resolve/e665e9ca7d823caa541e4de3acc046467c910eae/templates/llama.txt HTTP/1.1" 200 0
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  8.66it/s]
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/McGill-NLP/weblinx HTTP/1.1" 307 67
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/McGill-NLP/WebLINX HTTP/1.1" 200 2412
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/McGill-NLP/weblinx/McGill-NLP/weblinx.py HTTP/1.1" 404 0
DEBUG:urllib3.connecti

In [21]:
print(len(test_dataset))
print(len(valid_dataset))

2308
2126


In [13]:
#separate into halves for bugfixing + seeing if things went wrong
first_half_valid_texts = valid_turn_text[:len(valid_turn_text)//2]
second_half_valid_texts = valid_turn_text[len(valid_turn_text)//2:]

In [ ]:
llama1_3_action_model = pipeline("text-generation",
    model="McGill-NLP/Sheared-LLaMA-1.3B-weblinx", device=device, torch_dtype='auto'
)

In [12]:
llama1_3_first_half_valid_correct = []
llama1_3_first_half_valid_predictions = []
#collect correct and predictions from the model / data
llama1_3_valid_first_outputs = llama1_3_action_model(first_half_valid_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama1_3_valid_first_outputs):
    pred = out[0]['generated_text']
    llama1_3_first_half_valid_correct.append(valid_turn_info[i]["action"])
    llama1_3_first_half_valid_predictions.append(pred)

In [14]:
llama1_3_second_half_valid_correct = []
llama1_3_second_half_valid_predictions = []
llama1_3_valid_second_outputs = llama1_3_action_model(second_half_valid_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama1_3_valid_second_outputs):
    pred = out[0]['generated_text']
    llama1_3_second_half_valid_correct.append(valid_turn_info[i + len(valid_turn_text)//2]["action"])
    llama1_3_second_half_valid_predictions.append(pred)

In [15]:
# save the correct and predictions as jsons for later usage
llama1_3_valid_correct = llama1_3_first_half_valid_correct + llama1_3_second_half_valid_correct
llama1_3_valid_predictions = llama1_3_first_half_valid_predictions + llama1_3_second_half_valid_predictions
file_path = "llama1_3_valid_correct.json"

with open(file_path, "w") as file:
    json.dump(llama1_3_valid_correct, file)

print("List has been saved to", file_path)

file_path2 = "llama1_3_valid_predictions.json"

with open(file_path2, "w") as file:

    json.dump(llama1_3_valid_predictions, file)

print("List has been saved to", file_path2)


List has been saved to llama1_3_valid_correct.json
List has been saved to llama1_3_valid_predictions.json


In [14]:
#do the same for test dataset as valid
first_half_test_texts = test_turn_text[:len(test_turn_text)//2]
second_half_test_texts = test_turn_text[len(test_turn_text)//2:]

In [ ]:
llama1_3_first_half_test_correct = []
llama1_3_first_half_test_predictions = []
llama1_3_test_first_outputs = llama1_3_action_model(first_half_test_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama1_3_test_first_outputs):
    pred = out[0]['generated_text']
    llama1_3_first_half_test_correct.append(test_turn_info[i]["action"])
    llama1_3_first_half_test_predictions.append(pred)

In [ ]:
llama1_3_second_half_test_correct = []
llama1_3_second_half_test_predictions = []
llama1_3_test_second_outputs = llama1_3_action_model(second_half_test_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama1_3_test_second_outputs):
    pred = out[0]['generated_text']
    llama1_3_second_half_test_correct.append(test_turn_info[i + len(test_turn_text)//2]["action"])
    llama1_3_second_half_test_predictions.append(pred)

In [ ]:
llama1_3_test_correct = llama1_3_first_half_test_correct + llama1_3_second_half_test_correct
llama1_3_test_predictions = llama1_3_first_half_test_predictions + llama1_3_second_half_test_predictions
file_path = "llama1_3_test_predictions.json"

with open(file_path, "w") as file:
    json.dump(llama1_3_test_predictions, file)

print("List has been saved to", file_path)

file_path2 = "llama1_3_test_correct.json"

with open(file_path2, "w") as file:
    json.dump(llama1_3_test_correct, file)

print("List has been saved to", file_path2)

In [ ]:
#Now do it all for LLaMA2.7!
llama2_7_action_model = pipeline("text-generation",
    model="McGill-NLP/Sheared-LLaMA-2.7B-weblinx", device=device, torch_dtype='auto'
)

In [ ]:
llama2_7_first_half_valid_correct = []
llama2_7_first_half_valid_predictions = []
llama2_7_valid_first_outputs = llama2_7_action_model(first_half_valid_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama2_7_valid_first_outputs):
    pred = out[0]['generated_text']
    llama2_7_first_half_valid_correct.append(valid_turn_info[i]["action"])
    llama2_7_first_half_valid_predictions.append(pred)

In [ ]:
llama2_7_second_half_valid_correct = []
llama2_7_second_half_valid_predictions = []
llama2_7_valid_second_outputs = llama2_7_action_model(second_half_valid_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama2_7_valid_second_outputs):
    pred = out[0]['generated_text']
    llama2_7_second_half_valid_correct.append(valid_turn_info[i + len(valid_turn_text)//2]["action"])
    llama2_7_second_half_valid_predictions.append(pred)

In [ ]:
llama2_7_valid_correct = llama2_7_first_half_valid_correct + llama2_7_second_half_valid_correct
llama2_7_valid_predictions = llama2_7_first_half_valid_predictions + llama2_7_second_half_valid_predictions
file_path = "llama2_7_valid_correct.json"

with open(file_path, "w") as file:
    json.dump(llama2_7_valid_correct, file)

print("List has been saved to", file_path)

file_path2 = "llama2_7_valid_predictions.json"

with open(file_path2, "w") as file:
    json.dump(llama2_7_valid_predictions, file)

print("List has been saved to", file_path2)

In [ ]:
llama2_7_first_half_test_correct = []
llama2_7_first_half_test_predictions = []
llama2_7_test_first_outputs = llama2_7_action_model(first_half_test_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama2_7_test_first_outputs):
    pred = out[0]['generated_text']
    llama2_7_first_half_test_correct.append(test_turn_info[i]["action"])
    llama2_7_first_half_test_predictions.append(pred)

In [ ]:
llama2_7_second_half_test_correct = []
llama2_7_second_half_test_predictions = []
llama2_7_test_second_outputs = llama2_7_action_model(second_half_test_texts, return_full_text=False, max_new_tokens=64, truncation=True)
for i, out in enumerate(llama2_7_test_second_outputs):
    pred = out[0]['generated_text']
    llama2_7_second_half_test_correct.append(test_turn_info[i + len(test_turn_text)//2]["action"])
    llama2_7_second_half_test_predictions.append(pred)

In [ ]:
llama2_7_test_correct = llama2_7_first_half_test_correct + llama2_7_second_half_test_correct
llama2_7_test_predictions = llama2_7_first_half_test_predictions + llama2_7_second_half_test_predictions
file_path = "llama2_7_test_predictions.json"

with open(file_path, "w") as file:
    json.dump(llama2_7_test_predictions, file)

print("List has been saved to", file_path)

file_path2 = "llama2_7_test_correct.json"

with open(file_path2, "w") as file:
    json.dump(llama2_7_test_correct, file)

print("List has been saved to", file_path2)

In [ ]:
file_names = ["llama1_3_test_correct.json", "llama1_3_test_predictions.json", "llama1_3_valid_correct.json", "llama1_3_valid_predictions.json"]

variable_names = ["llama1_3_test_correct", "llama1_3_test_predictions", "llama1_3_valid_correct", "llama1_3_valid_predictions"]

llama1_3_data_dict = {}


for file_name, var_name in zip(file_names, variable_names):
    path_to_file = os.path.join("experiment_data", file_name)
    with open(path_to_file, "r") as file:
        data_list = json.load(file)
        llama1_3_data_dict[var_name] = data_list

print("Contents of the dictionary:")
for var_name, data_list in llama1_3_data_dict.items():
    print(f"{var_name}: {data_list}")

In [ ]:
# Get all of the cleaned up predictions and correct actions
llama1_3_valid_correct_cleaned = []
llama1_3_valid_predictions_cleaned = []
for corr, pred in zip(llama1_3_data_dict["llama1_3_valid_correct"], llama1_3_data_dict["llama1_3_valid_predictions"]):
    #get the action predicted
    closing_paren_index = pred.find(')')
    substring = pred[:closing_paren_index + 1]  # Include the closing parenthesis
    #clean up the leading and trailing blanks for correct and predicted, lower, and get rid of punctuation 
    pred_cleaned = substring.strip().translate(str.maketrans('', '', string.punctuation)).lower()
    corr_cleaned = corr.translate(str.maketrans('', '', string.punctuation)).lower()
    llama1_3_valid_correct_cleaned.append(corr_cleaned)
    llama1_3_valid_predictions_cleaned.append(pred_cleaned)
print(llama1_3_valid_correct_cleaned)
print(llama1_3_valid_predictions_cleaned)

In [ ]:
# Get all of cleaned up actions but for test this time
llama1_3_test_correct_cleaned = []
llama1_3_test_predictions_cleaned = []
for corr, pred in zip(llama1_3_data_dict["llama1_3_test_correct"], llama1_3_data_dict["llama1_3_test_predictions"]):
    closing_paren_index = pred.find(')')
    substring = pred[:closing_paren_index + 1]
    pred_cleaned = substring.strip().translate(str.maketrans('', '', string.punctuation)).lower()
    corr_cleaned = corr.translate(str.maketrans('', '', string.punctuation)).lower()
    llama1_3_test_correct_cleaned.append(corr_cleaned)
    llama1_3_test_predictions_cleaned.append(pred_cleaned)
print(llama1_3_test_correct_cleaned)
print(llama1_3_test_predictions_cleaned)

In [ ]:
# do the same for llama 2.7B
file_names = ["llama2_7_test_correct.json", "llama2_7_test_predictions.json", "llama2_7_valid_correct.json", "llama2_7_valid_predictions.json"]

variable_names = ["llama2_7_test_correct", "llama2_7_test_predictions", "llama2_7_valid_correct", "llama2_7_valid_predictions"]

llama2_7_data_dict = {}

for file_name, var_name in zip(file_names, variable_names):
    path_to_file = os.path.join("experiment_data", file_name)
    with open(path_to_file, "r") as file:
        data_list = json.load(file)
        llama2_7_data_dict[var_name] = data_list

print("Contents of the dictionary:")
for var_name, data_list in llama2_7_data_dict.items():
    print(f"{var_name}: {data_list}")

In [ ]:
llama2_7_valid_correct_cleaned = []
llama2_7_valid_predictions_cleaned = []
for corr, pred in zip(llama2_7_data_dict["llama2_7_valid_correct"], llama2_7_data_dict["llama2_7_valid_predictions"]):
    closing_paren_index = pred.find(')')
    substring = pred[:closing_paren_index + 1]
    pred_cleaned = substring.strip().translate(str.maketrans('', '', string.punctuation)).lower()
    corr_cleaned = corr.translate(str.maketrans('', '', string.punctuation)).lower()
    llama2_7_valid_correct_cleaned.append(corr_cleaned)
    llama2_7_valid_predictions_cleaned.append(pred_cleaned)
print(llama2_7_valid_correct_cleaned)
print(llama2_7_valid_predictions_cleaned)

In [ ]:
llama2_7_test_correct_cleaned = []
llama2_7_test_predictions_cleaned = []
for corr, pred in zip(llama2_7_data_dict["llama2_7_test_correct"], llama2_7_data_dict["llama2_7_test_predictions"]):
    closing_paren_index = pred.find(')')
    substring = pred[:closing_paren_index + 1]
    pred_cleaned = substring.strip().translate(str.maketrans('', '', string.punctuation)).lower()
    corr_cleaned = corr.translate(str.maketrans('', '', string.punctuation)).lower()
    llama2_7_test_correct_cleaned.append(corr_cleaned)
    llama2_7_test_predictions_cleaned.append(pred_cleaned)
print(llama2_7_test_correct_cleaned)
print(llama2_7_test_predictions_cleaned)

In [64]:
def calculate_accuracy(list1, list2, threshold):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of the same length")
    hits = 0
    total = len(list1)
    similarities = []
    for str1, str2 in zip(list1, list2):
        # Calculate Jaro-Winkler similarity directly
        similarity = jellyfish.jaro_winkler_similarity(str1, str2)
        
        # If similarity is greater than the threshold, or one of the strings contains the other, count it as a hit
        if similarity >= threshold or str1 in str2 or str2 in str1:
            hits += 1
        similarities.append(similarity)
    # get average jaro-winkler score
    avg_score = np.mean(similarities) * 100
    accuracy = (hits / total) * 100
    return accuracy, avg_score

In [78]:
# All of the values below this are from the local prediction and correct jsons. Feel free to create your own using the above code to get your own numbers!
# Also, due to the running time and computational load that running the models carries, we ran it and got the jsons once, because it was already a lot of data.
threshold = 0.8
llama1_3_valid_accuracy, llama1_3_valid_jaro_score = calculate_accuracy(llama1_3_valid_predictions_cleaned, llama1_3_valid_correct_cleaned, threshold)
print(f"Accuracy of LLaMA1.3B on Valid Dataset for Jaro-Winkler similarity >= {threshold:.2f}: {llama1_3_valid_accuracy:.2f}%")
print(f"Average Jaro-Winkler Score of LLaMA1.3B on Valid Dataset: {llama1_3_valid_jaro_score:.2f}%")

Accuracy of LLaMA1.3B on Valid Dataset for Jaro-Winkler similarity >= 0.80: 56.35%
Average Jaro-Winkler Score of LLaMA1.3B on Valid Dataset: 66.37%


In [79]:
threshold = 0.8
llama1_3_test_accuracy, llama1_3_test_jaro_score = calculate_accuracy(llama1_3_test_predictions_cleaned, llama1_3_test_correct_cleaned, threshold)
print(f"Accuracy of LLaMA1.3B on test Dataset for Jaro-Winkler similarity >= {threshold:.2f}: {llama1_3_test_accuracy:.2f}%")
print(f"Average Jaro-Winkler Score of LLaMA1.3B on test Dataset: {llama1_3_test_jaro_score:.2f}%")

Accuracy of LLaMA1.3B on test Dataset for Jaro-Winkler similarity >= 0.80: 56.85%
Average Jaro-Winkler Score of LLaMA1.3B on test Dataset: 67.33%


In [80]:
threshold = 0.8
llama2_7_valid_accuracy, llama2_7_valid_jaro_score = calculate_accuracy(llama2_7_valid_predictions_cleaned, llama2_7_valid_correct_cleaned, threshold)
print(f"Accuracy of LLaMA1.3B on Valid Dataset for Jaro-Winkler similarity >= {threshold:.2f}: {llama2_7_valid_accuracy:.2f}%")
print(f"Average Jaro-Winkler Score of LLaMA2.7B on Valid Dataset: {llama2_7_valid_jaro_score:.2f}%")

Accuracy of LLaMA1.3B on Valid Dataset for Jaro-Winkler similarity >= 0.80: 60.02%
Average Jaro-Winkler Score of LLaMA2.7B on Valid Dataset: 70.57%


In [81]:
threshold = 0.8
llama2_7_test_accuracy, llama2_7_test_jaro_score = calculate_accuracy(llama2_7_test_predictions_cleaned, llama2_7_test_correct_cleaned, threshold)
print(f"Accuracy of LLaMA1.3B on test Dataset for Jaro-Winkler similarity >= {threshold:.2f}: {llama2_7_test_accuracy:.2f}%")
print(f"Average Jaro-Winkler Score of LLaMA2.7B on test Dataset: {llama2_7_test_jaro_score:.2f}%")

Accuracy of LLaMA1.3B on test Dataset for Jaro-Winkler similarity >= 0.80: 46.01%
Average Jaro-Winkler Score of LLaMA2.7B on test Dataset: 62.77%
